In [2]:
from huggingface_hub import login

login()

In [3]:
!pip install -U bitsandbytes

In [28]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load Llama3-8B model & tokenizer
model_name = "meta-llama/Llama-3.2-3B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [29]:
from transformers import pipeline

llama_pipeline = pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    tokenizer=tokenizer,
    device_map="auto",
)

Device set to use cuda:0


In [31]:
def get_llama_response(prompt):
    sequences = llama_pipeline(
        prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=512,
    )
    return sequences[0]['generated_text']

In [40]:
base_prompt = 'Based on the context above, guess the company name'
def infer_company_name(context):
    prompt = f"{context}\n\n{base_prompt}\nAnswer:"
    full_response = get_llama_response(prompt)
    answer_marker = "Answer:"
    if answer_marker in full_response:
        guess = full_response.split(answer_marker, 1)[-1].strip()
    else:
        guess = full_response.strip()

    return guess

In [41]:
# Ground truth - Tesla
masked_text = 'The company reported strong earnings for Q1, with revenue growth exceeding analyst expectations. The new product launch in time T was well received, driving stock prices higher.'

In [44]:
g = infer_company_name(masked_text)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [45]:
g

'Apple'